In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [16]:
img_width, img_height = 299, 299
batch_size = 32
epochs = 10

In [17]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [18]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [19]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [20]:
for layer in base_model.layers:
    layer.trainable = False

In [21]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=[Accuracy()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0         
                                                                 
 dense_3 (Dense)             (None, 256)               33554688  
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 55390497 (211.30 MB)
Trainable params: 33587713 (128.13 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [23]:
checkpoint_path = '/content/drive/MyDrive/InceptionV3.h5'
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [24]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 146.9731 - accuracy: 0.6094
Epoch 1: val_loss improved from inf to 131.73370, saving model to /content/drive/MyDrive/InceptionV3.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 237s 11s/step - loss: 146.9731 - accuracy: 0.6094 - val_loss: 131.7337 - val_accuracy: 0.5437
Epoch 2/10
20/20 [==============================] - ETA: 0s - loss: 34.0607 - accuracy: 0.7547
Epoch 2: val_loss improved from 131.73370 to 24.26396, saving model to /content/drive/MyDrive/InceptionV3.h5
20/20 [==============================] - 278s 14s/step - loss: 34.0607 - accuracy: 0.7547 - val_loss: 24.2640 - val_accuracy: 0.6187
Epoch 3/10
20/20 [==============================] - ETA: 0s - loss: 10.0051 - accuracy: 0.7812
Epoch 3: val_loss improved from 24.26396 to 18.26864, saving model to /content/drive/MyDrive/InceptionV3.h5
20/20 [==============================] - 226s 11s/step - loss: 10.0051 - accuracy: 0.7812 - val_loss: 18.2686 - val_accuracy: 0.5625
Epoch 4/10
20/20 [==============================] - ETA: 0s - loss: 10.0710 - accuracy: 0.7531
Epoch 4: val_loss did not improve from 18.26864
20/20 [==============================] - 268s 13s

In [25]:
model = models.load_model(checkpoint_path)

In [26]:
model.save('/content/drive/MyDrive/InceptionV3.h5')